# Dataset
* Use Stanford dog dataset with over 20k images of dogs, but only use 4096 images.
* Resize images to shape 12288 (64*64*3).
* Copy all resized images to numpy array of shape (12288, 4096).
* Any prediction above 50% is good!!!

In [ ]:
import kagglehub

# Download latest version of stanford dog dataset
path_dogs_dataset = kagglehub.dataset_download("jessicali9530/stanford-dogs-dataset")

# Download latest version of stanford car dataset
path_cars_dataset = kagglehub.dataset_download("jessicali9530/stanford-cars-dataset")

In [ ]:
# Store images and targets to hdf5 format

import numpy as np
from PIL import Image
import glob
import os
import h5py

np.random.seed(18)
SHAPE_IMAGES = (64, 64)

def generate_dataset(directory, shape=(64, 64), num_img=4096):
    image_paths = glob.glob(os.path.join(directory, '**', '*.*'), recursive=True)
    image_paths = [p for p in image_paths if p.lower().endswith(('.jpeg', '.jpg'))]
    images = []
    for path in image_paths:
        if len(images) >= num_img:
            break
        img = Image.open(path).convert('RGB').resize(shape)  # Convert to RGB and resize
        images.append(np.array(img).flatten())  # Flatten into a 12288 (64x64x3) array

    return np.array(images).T  # Shape 12288 x NUM_IMAGES

Xdogs = generate_dataset(path_dogs_dataset, shape=SHAPE_IMAGES)
Xcars = generate_dataset(path_cars_dataset, shape=SHAPE_IMAGES)
X = np.hstack((Xdogs, Xcars)) / 255. # Normalize data from 0.0 to 1.0
Ydogs = np.ones((1, Xdogs.shape[1]), dtype='int8')
Ycars = np.zeros((1, Xcars.shape[1]), dtype='int8')
Y = np.hstack((Ydogs, Ycars))

shuffled_indices = np.random.permutation(X.shape[1])
X = X[:, shuffled_indices]
Y = Y[:, shuffled_indices]

split_index = int(0.8 * X.shape[1])
Xtrain, Xtest = X[:, :split_index], X[:, split_index:]
Ytrain, Ytest = Y[:, :split_index], Y[:, split_index:]

# Open the HDF5 file for writing (creates the file if it doesn't exist)
with h5py.File('dogs.h5', 'w') as hf:
    hf.create_dataset('Xtrain', data=Xtrain)
    hf.create_dataset('Xtest', data=Xtest)
    hf.create_dataset('Ytrain', data=Ytrain)
    hf.create_dataset('Ytest', data=Ytest)

# Training
* A single node made of a matrix W of shape (12288, 1) and a bias b
* Initialize W and b to 0

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import h5py
import numpy as np
import time

def load_dataset(shape_img=(64, 64), show_samples=False):
    with h5py.File('dogs.h5', 'r') as hf:
        Xtrain = hf['Xtrain'][:].astype(np.float32)
        Xtest = hf['Xtest'][:].astype(np.float32)
        Ytrain = hf['Ytrain'][:].astype(np.int8)
        Ytest = hf['Ytest'][:].astype(np.int8)
    
    if show_samples:
        # Plot first 5 images of each dataset for verification
        fig, axes = plt.subplots(2, 10, figsize=(20, 4))
        for i in range(10):
            x = Xtrain[:, i].reshape(shape_img[0], shape_img[1], 3)
            axes[0, i].imshow(x)
            axes[0, i].axis('off')  # Hide the axis
            axes[0, i].set_title("Dog" if Ytrain[0, i] == 1 else "Car", fontsize=8)
            x = Xtest[:, i].reshape(shape_img[0], shape_img[1], 3)
            axes[1, i].imshow(x)
            axes[1, i].axis('off')  # Hide the axis
            axes[1, i].set_title("Dog" if Ytest[0, i] == 1 else "Car", fontsize=8)

        plt.show()

    return Xtrain, Xtest, Ytrain, Ytest

def linear_combination(X, W, b):
    Z = np.dot(W.T, X) + b

    return Z

def sigmoid(Z): # Compute the sigmoid of z
    A = 1.0 / (1.0 + np.exp(-Z))
    
    return A

def compute_cost(Y, A):
    J = -np.sum(Y * np.log(A) + (1.0 - Y) * np.log(1.0 - A)) / A.shape[1]
    
    return J

def compute_accuracy(Xtrain, Xtest, Ytrain, Ytest, W, b): # Compute accuracy: (True Positives + True Negatives) / Total predictions
    Ypred_train = predict(Xtrain, W, b)
    Ypred_test = predict(Xtest, W, b)
    
    acc_train = 100 - np.mean(np.abs(Ypred_train - Ytrain)) * 100
    acc_test = 100 - np.mean(np.abs(Ypred_test - Ytest)) * 100

    return acc_train, acc_test

def compute_recall(Xtrain, Xtest, Ytrain, Ytest, W, b): # Compute accuracy: True Positives / (True Positives + False Negatives)
    Ypred_train = predict(Xtrain, W, b)
    Ypred_test = predict(Xtest, W, b)
    
    TP = np.sum((Ytrain == 1) & (Ypred_train == 1)) # True Positives: Actual dog (1) correctly predicted as dog (1)
    FN = np.sum((Ytrain == 1) & (Ypred_train == 0)) # False Negatives: Actual dog (1) predicted as not dog (0)
    recall_train = TP / (TP + FN) if (TP + FN) > 0 else 0   # Recall calculation

    TP = np.sum((Ytest == 1) & (Ypred_test == 1)) # True Positives: Actual dog (1) correctly predicted as dog (1)
    FN = np.sum((Ytest == 1) & (Ypred_test == 0)) # False Negatives: Actual dog (1) predicted as not dog (0)
    recall_test = TP / (TP + FN) if (TP + FN) > 0 else 0   # Recall calculation

    return 100. * recall_train, 100. * recall_test

def predict(X, W, b, labels=None, shape_img=(64, 64), show_samples=False):
    _, m = X.shape

    # Forward propagation
    Z = linear_combination(X, W, b) # Z = (1, m)
    A = sigmoid(Z) # A = (1, m)

    Ypred = np.zeros((1, m))
    Ypred = np.where(A > 0.5, 1, 0)

    if show_samples:
        # Plot first 10 predictions
        fig, axes = plt.subplots(1, 10, figsize=(20, 4))
        for i in range(10):
            x = X[:, i].reshape(shape_img[0], shape_img[1], 3)
            axes[i].imshow(x)
            axes[i].axis('off')  # Hide the axis
            title = f'GT: {labels[Ytest[0, i]]}\nPred: {labels[Ypred[0, i]]}'
            axes[i].set_title(title)

        plt.show()
    
    return Ypred

def plot_cost_function(performance_metrics, lr = 0.01):
    costs = [metrics[0] for metrics in performance_metrics]
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title(f'Cost Function over Iteration\nLearning rate = {lr}')
    plt.show()
    
def plot_accuracy(performance_metrics, lr = 0.01):
    accs_train = [metrics[1] for metrics in performance_metrics]
    accs_test = [metrics[2] for metrics in performance_metrics]
    fig, ax1 = plt.subplots()

    # Plot accs_train on the left y-axis
    ax1.plot(accs_train, 'b-', label='Train Accuracy')
    ax1.set_xlabel('iterations (per hundreds)')
    ax1.set_ylabel('Train Accuracy [%]', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    # Set x-axis to logarithmic scale
    ax1.set_xscale('log')

    # Create a second y-axis for accs_test
    ax2 = ax1.twinx()
    ax2.plot(accs_test, 'r-', label='Test Accuracy')
    ax2.set_ylabel('Test Accuracy [%]', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # Add a title and show the plot
    plt.title(f'Accuracy\nLearning rate = {lr}')
    fig.tight_layout()
    plt.show()

def plot_recall(performance_metrics, lr = 0.01):
    recall_train = [metrics[3] for metrics in performance_metrics]
    recall_test = [metrics[4] for metrics in performance_metrics]
    fig, ax1 = plt.subplots()

    # Plot recall_train on the left y-axis
    ax1.plot(recall_train, 'b-', label='Train Recall')
    ax1.set_xlabel('iterations (per hundreds)')
    ax1.set_ylabel('Train Recall [%]', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    # Set x-axis to logarithmic scale
    ax1.set_xscale('log')

    # Create a second y-axis for recall_test
    ax2 = ax1.twinx()
    ax2.plot(recall_test, 'r-', label='Test Recall')
    ax2.set_ylabel('Test Recall [%]', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

    # Add a title and show the plot
    plt.title(f'Recall\nLearning rate = {lr}')
    fig.tight_layout()
    plt.show()

def plot_performance(performance_metrics, lr = 0.01):
    plot_cost_function(performance_metrics, lr)
    plot_accuracy(performance_metrics, lr)
    plot_recall(performance_metrics, lr)

def show_saliency_map(W, b):
    X = np.ones((64 * 64 * 3, 1))

    Z = W * X + b
    A = sigmoid(Z) 
    A = (255.0 * (A - np.min(A)) / (np.max(A) - np.min(A))).astype(np.uint8)
    img = A.reshape(64, 64, 3)

    colors = ['Reds', 'Greens', 'Blues']
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes[0, 0].imshow(img)
    axes[0, 0].axis('off')  # Hide the axis
    axes[0, 0].set_title('Weight-Based Saliency Color Map for Cat vs. Car Classification')
    for i in range(3):
        axes[1, i].imshow(img[:, :, i], cmap=colors[i], norm=Normalize(vmin=0, vmax=255))
        axes[1, i].axis('off')  # Hide the axis
        axes[1, i].set_title(f'{colors[i]}')

    plt.show()

In [ ]:
class LogisticRegression:
    def __init__(self, dataset, lr=0.00001, iter=10000, labels=['Car', 'Dog'], params_init='min', invert_labels=False):
        self.Xtrain, self.Xtest, self.Ytrain, self.Ytest = dataset
        self.nx, self.m = self.Xtrain.shape
        if invert_labels == True:
            self.Ytrain = 1. - self.Ytrain
            self.Ytest = 1. - self.Ytest
        self.lr = lr
        self.iter = iter
        self.labels = labels
        self.params_init = params_init
        self.invert_labels = invert_labels
        self.W, self.b = self.init_parameters(self.nx, self.params_init) # W = (n, 1)
        self.train()

    def init_parameters(self, nx, min_or_max):
        if min_or_max == 'min':
            W = np.zeros((nx, 1), dtype='float32')
        else:
            W = np.ones((nx, 1), dtype='float32')
        b = 0.0

        return W, b

    def train(self):
        t = time.time()
        performance_metrics = []
        for i in range(self.iter):
            # Forward propagation
            Z = linear_combination(self.Xtrain, self.W, self.b) # Z = (1, m)
            A = sigmoid(Z) # A = (1, m)

            # Compute cost function and model's accuracy
            if i % 100 == 0:
                J = compute_cost(self.Ytrain, A)
                acc_train, acc_test = compute_accuracy(self.Xtrain, self.Xtest, self.Ytrain, self.Ytest, self.W, self.b)
                recall_train, recall_test = compute_recall(self.Xtrain, self.Xtest, self.Ytrain, self.Ytest, self.W, self.b)
                performance_metrics.append((J, acc_train, acc_test, recall_train, recall_test))
                print(f'Iteration {i} out of {self.iter}')

            # Backward propagation
            dW = np.dot(self.Xtrain, (A - self.Ytrain).T) / self.m # dW = (n, 1)
            db = np.sum(A - self.Ytrain) / self.m
            self.W -= self.lr * dW
            self.b -= self.lr * db
            
            print(f'time: {((time.time() - t)*1000.):.1f}ms')
            t = time.time()

        plot_performance(performance_metrics, self.lr)
        # show_saliency_map(W, b)


In [ ]:
dataset = load_dataset(show_samples=True)

model4 = LogisticRegression(dataset, lr=0.00001, iter=10000, params_init='max', invert_labels=True)

model1 = LogisticRegression(dataset, lr=0.00001, iter=10000, params_init='min', invert_labels=False)

model2 = LogisticRegression(dataset, lr=0.00001, iter=10000, params_init='min', invert_labels=True)

model3 = LogisticRegression(dataset, lr=0.00001, iter=10000, params_init='max', invert_labels=False)



'''if INVERT_LABELS == True:
    print('Labels: 0 for dogs, 1 for cars')
else:
    print('Labels: 1 for dogs, 0 for cars')
if PARAMS_INIT == 'min':
    print('Set parameters to zero')
else:
    print('Set parameters to one')
print(f'  Accuracy: train = {acc_train:.1f}%, test = {acc_test:.1f}%')
print(f'  Recall: train = {recall_train:.1f}%, test = {recall_test:.1f}%\n')'''

# print(W)
# print(b)

* True Positive (TP): The model correctly predicted that there was a dog in the image when there actually was a dog.
* True Negative (TN): The model correctly predicted that there was no dog in the image when there actually was no dog.
* False Positive (FP): The model incorrectly predicted a dog in the image when there actually was no dog (this is often called a "Type I error").
* False Negative (FN): The model incorrectly predicted no dog in the image when there actually was a dog (this is often called a "Type II error").
* Accuracy = (True Positives + True Negatives) / Total predictions
* Recall = True Positives / (True Positives + False Negatives)